### Feature based approaches to Activity Recognition
Dense trajectories and motion boundary descriptors for action recognition:<br>
What we do is we have this video that's playing and we are going to detect these key points that are good to track in the video. and then we are going to tracking them and end up with all these little track trajectories that we actually track across the video and lots of feature about those trajectories and about the surrounding features get accumulatted enough histograms
![Dense trajectories](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Dense_trajectories.png)
Basiclly you can't track feature other regioins inthe viedeo that are too smooth because you can't lock onto any visual feature and they are ways for basically getting a set of points that are easy to tack in the video<br>
optical flow method: basically give you a motion filed from one frame to another and tell you how the schema moved from one frame to next frame. It will take a frame and a next frame and will solve for a motion field a displacement vector at every single position into where it traveled or how the frame moved.<br>
Trajectory可以用向量来表示，向量中的每个值代表在一个frame上的该点的[x,y]坐标。
Action Recognition with Improved Trajectories<br>

### Spatio-Temporal ConvNets
一种方法是：一般图片会有3个channel，我们可以把不同的frame进行stack，比如有15个frame的话，我们可以对应的是channel为3*15。<br>
Extend the convolutional filters in time, perform spatio-temporal convolutions：说是增加time维，但它给的例子是11*11*T，
![Spatio Temporal ConvNets](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Spatio_Temporal_ConvNets.png)
![Spatio Temporal ConvNets](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Spatio_Temporal_ConvNets2.png)
注意区别Early Fusion和Slow Fusion，第一个方法是通过stack，把多个frame堆叠在一起，然后一起进行处理，it's kinds of like your filters on the very first conv layer have a large temporal extent but from then on everthign else is two-deimensional competition.这个不是很理解，居然第一层都扩展filter了，为什么后面又放弃了<br>
注意：有时候一个video中单张图片就能很好的处理问题，所以玩之前，认真的想想，你是否真的需要这个video。
另外推荐的结构有：
Two-stream Convolutional Networks for Action Recognition in Videos 2014<br>
Features with 3D Convolutional Networks 2015

#### Long time Spatio Temporal ConvNets
前面提到的方法都只是关注局部的信息，当我们的video很长，并且其中不同事件相互影响的时候，我们应该怎么进行处理呢？？<br>
一货提到，可以模仿图片的multi scale，处理不同播放倍速的video。一种推荐的方法是使用LSTM
![ltstcn](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Long_time_Spatio_temporal_convNet.png)
还有一个问题，原话是：<br>
There is kind of this ugly asymmetry about these components. So you have these part neurons inside the 3d convNet are function of some small local chunk of video and then you have these neurons on the very top that are functions of everything in the video(RNN)<br>
The idea is that we are going to get rid of the RNN, we're going to basically take a convNet and we're going to make every single neuron in that convNet be a small recurrent neural network like every single neuron becomes recurrent in the convNet. What we'll do instead is, we're going to make every single convolution layer a kind of recurrent layer, so the way we do that is just as before we take the input from the below and we do conv over it but we also take our previous output from the previous time step of this conv layer output. and we do convolution over both to produce our output. It is very beautiful because there are only use two dimentional convolution here.
![ltstcn](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/RNN_ConvNet.png)

### Unsupervised Learning
仅记录感兴趣的<br>
- Autoencoder的encoder和decoder sometime share weights
- We learned the encoder network which hopefully from all this unsupervised data has learned to compress the data and extract some useful features and then we're going to use this encoder network to initialize part of the large supervised netwok and now when we acutally do have access to maybe some smaller data that has some labels then the most of the work here could have been done by this unsupervised training at the beginning and then we can just use that to initialize the bigger network and then fine to the whole thing with hopefully a very small amount of supervised data 实际效果感人。
- Greedy training: with boltzmann machine

#### Variational Autoencoder
Wit Variational Autoencoder we assuming this particluar type of method by which our data was generated. We assume that there exist out there in the world some prior distribution generating these latent State Z. and we assume some conditional distribution that once we have the latent state we can generate sample from some other distribution to generate the data. So the variational autoencoder really imageines that our data was generated by these pretty simple process. And the intuition is that X is something like the image and Z maybe summarizes some useful stuff about that image(e.g class) <br>
So the problem now we want to estimate the parameters $\theta$ of both the prior and conditional without actually having access to these latent States Z.<br>
suppose that we have the laten state Z for some piece of data and we assume that latent data will go into some decoder network which could be some big complicated neural network. and now that neural network is going to spit out two things. It's going to spit out the mean of the data X and also the variance of the data X. It's look very like the top half of a normal autoencoder that we have this laten state we have some neural net that's operating on the latent state but now instead of directly spitting out the data, it spitting out the mean and variance of the data.<br>
So now the problem is that, by bayes rule if given the prior and the conditional bayer tells us the posterior. So if we acutally want to use this model, we need to be able to estimate the latent state from the input data and the way that we estimate the latent state from the input data is writing down this posterior distribution which is the probability of the latent state Z given our observed data X<br>
We have access to the prior because we assumed its unit Gaussian <br>
This encoder network is going to take in the data point and it's going to spit out a distribution over the latent state space. again this looks very much the same as the bottom half of a traditioinal autoencoder where we taking in the data and now instead of directly spitting out the latent, we are going to spit out a mean and a covariance of the latent state and again this encoder network might be something fully convolutional network or it might be some deep convolutional network. So sort of the intuition of that will be that this encoder network will be this separate totally different disjoint function but we're going to train it in a way so that it approximates this posterior distribution that we don't acutally have access to.<br>
Once we stick this all together, we have these input data point X we're going to pass it through our encoder network and the encoder will spit out the distribution over the latent States. Once we have this distribution about these latent state, you can imagine sampling from that distribution to get some latent state of high probability for that input. Then once we have some concrete example of a latent state then we can pass it through the decoder network which will spit out the probability of the data again. And then we have this distribution of the data. we can sample it to get something that hopefully looks like the original data point <br>
When we training this thing, it use actually very similar method as a normal autoencode where we have this forward pass and this backward pass the only different is on the lost function. So at the top we have this reconstruction loss rather than being this point wise l2 instead we want this distribution to the true input data and we also have these loss term coming in the middle that we want this generated distribution over the laten states to very similar to our stated prior distribution.
![variational autoencoder](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Variational_Autoencoder_encoder.png)
![variational autoencoder](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Variational_Autoencoder_decoder.png)
![variational autoencoder](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Variational_Autoencoder_training.png)
![variational autoencoder](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/Variational_Autoencoder_generate_data.png)
课程上还有数学解释，可以看看

### Generative Adversarial Nets
![Semantic_Segmentation](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/GAN.png)
generate data explicitly without having to worry about divergences and priors and the sort of stuff.<br>
The idea is that we are going to have a generative network. We are going to start some random noise that probably is drawn from like a unit Gaussian or something like that and then we are going to have a generator network and this generator network actually looks very much like the decoder network in variational autoencoder. In that we are going to take this random noise and we are going to spit out an image that is going to be some fake non real image that just generating using this train network.<br>
Then we are also going to hook up a discriminator network, that is going to look at this fake image and try ot decide weather or not that the generated image is fake. So the second network is just doing this binary classification task. <br>
Then we can train this thing all jointly where generator network will receive mini batch of random noise and it will spit out fake image and our discriminator network will receive mini batches of partially these fake images and partially real images from the dataset. and it will try to make this classification task to say which are real and which are fake.

#### multi scala
![Semantic_Segmentation](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/GAN_multi_scale_0.png)
![Semantic_Segmentation](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/GAN_multi_scale.png)

#### GAN: simplifying 
Generator is an upsampling network with gractionally-strided convolutions<br>
Discriminator is a convolutional network<br>
Architecture guidelines for stable Deep Convolutional GANs
- Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions(generator)
- Use batchnorm in both the generator and the discriminator
- Remove fully connected hidden layers for deeper architectures
- Use ReLU activation in generator for all layers except for the output, which uses Tanh 
- Use LeakyReLU activation in the discriminator for all layers.
![Semantic_Segmentation](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/GAN_simplifying.png)

#### Vector Math 
Generative Adversarial Nets: Vector Math Radford et al, ICLR 2016

#### putting everything together
![Semantic_Segmentation](https://raw.githubusercontent.com/HuangYiran/Convolutional_Neural_Networks/master/GAN_alltogether.png)

Object `numpy` not found.
